# Code-Switching Removal Pipeline - Groq API Testing

## Overview
This notebook translates **1,037 Hinglish/Marathi/code-mixed YouTube comments** to pure English using Groq's LLM API.

### What This Does
- **Input**: YouTube comments with code-switching (Hindi + English + Marathi mixed)
- **Output**: Pure English translations in new column `csw-removed-comment`
- **Purpose**: Remove linguistic noise to improve political bias classification

### Models & Prompts Being Tested
- **Model**:
    
    `"gpt-oss-120b": "openai/gpt-oss-120b",
    "llama3.3-70b": "llama-3.3-70b-versatile",
    "llama3.1-8b": "llama-3.1-8b-instant",
    "gpt-oss-20b": "openai/gpt-oss-20b"`
- **Prompt**: `political` (preserves political stance, sentiment, and cultural context)

### CSV Structure
- **Input columns**: `Serial Number`, `text`, `relabel_llm_output`, `llm_model`, `Label_Topic`
- **Output**: Same columns + new `csw-removed-comment` column

### Features
- Caching - Never translates the same comment twice  
- Incremental saves - Progress saved every 50 rows  
- Retry logic - Handles API failures automatically  
- Rate limiting - Respects Groq API limits  

---

**Expected runtime**: ~30-45 minutes for 1,037 comments  
**Cost**: ~$0.70 USD

---

### Setup Instructions
1. Set your Groq API key in the code below
2. Upload your CSV file
3. Run all cells
4. Download the translated CSV

In [ ]:
!pip install groq
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 2.9 MB/s eta 0:00:00


In [ ]:
"""
Quick Test Script for Groq Translation
"""

import os
from groq import Groq

# Set your API key (use Colab Secrets or hardcode temporarily)
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)



SAMPLE_COMMENTS = [
   "1:58 - Beta umar se pehly bara hogya :expressionless_face::expressionless_face:",
   "Arey porsche hai to paisa waale honge judge ko 4-5 cr de diya honge bas simple",
   "Rich people be like law mera la@@ pe.",
   "It's true what you said. If Justice doesn't happen here from it will promote mob justice which is even more terrible and Swift. Spot peh hi Justice deneka kya.",
   "Ok Rahulji to fir Bhopal gas durghatna main to hazaro log mare the tabhi Anderson farar kese hua tha ?  I am not protecting that kid who is minor but whatever baseless assessment this guy is giving is nonsense."
]



PROMPTS = {
    "default":
      """You are a translation model.
      Your task is to convert the following Hinglish, Marathi, or code-mixed YouTube comment text into fluent English that is clear and makes sense.

      Critical instructions:
      - Maintain the original meaning, sentiment, tone, political stance, and all context (including references to courts, judges, law, governance, and public safety).
      - Do NOT change, omit, or paraphrase—translate every part so that all meaning is preserved, even if the source comment sounds unusual or awkward.
      - Do NOT add explanations or any additional text.
      - The output sentence should make sense when spoken in English. It should sound semantically, emotionally and verbally similar to they way it was in the other language.
      - Make sure the emojies if there are any will be as test like ":expressionless face" make sure they stay the same in the output text./
      - Output ONLY the translated English comment as your response, matching the input structure.

      Input Comment: "{text}"

      """,

      "political":
        """You are translating comments related to Indian politics, judiciary, and public safety on YouTube.

        Translate this Hinglish, Marathi, or code-mixed comment into fluent and natural English that conveys the same message and sentiment, even if the comment appears odd or unconventional.

        Strict requirements:
        - Preserve the political stance (pro/anti government, pro/anti judiciary)
        - Maintain the original meaning, sentiment, tone, political stance, and all context (including references to courts, judges, law, governance, and public safety).
        - Do NOT change, omit, or paraphrase—translate every part so that all meaning is preserved, even if the source comment sounds unusual or awkward.
        - Do NOT add explanations or any additional text.
        - Retain all original sentiment intensity and tone (anger, support, neutrality, sarcasm)
        - Ensure the cultural context is intact (including references to relevant events, figures, or institutions)
        - Do NOT omit, add, or interpret—every word and nuance must be present in the translation
        - While not the case verythime when it end as a question, sometimes in indian languages, people tend to b rehtoric and say kya kyun or ? in the end of sentence, when they don't actually mean to add that. take care of such cases and make sure that they are statements and not questions for such unique cases.
        - The output sentence should make sense when spoken in English. It should sound semantically, emotionally and verbally similar to they way it was in the other language.
        - Make sure the emojies if there are any will be as test like ":expressionless face" make sure they stay the same in the output text./
        - Return ONLY the translated English comment in your response

        Input Comment: "{text}"

        """,

  }



MODELS = {
    "gpt-oss-120b": "openai/gpt-oss-120b",
    "llama3.3-70b": "llama-3.3-70b-versatile",
    "llama3.1-8b": "llama-3.1-8b-instant",
    "gpt-oss-20b": "openai/gpt-oss-20b"
}



def translate_groq(comment, model_id, prompt_template, temperature= 0.5):
  prompt = prompt_template.format(text=comment)

  try:
    response = client.chat.completions.create(
        model= model_id,
        messages= [
            {"role": "system", "content": "You're a linguistics expert with knowlegde of Hindi, Hinglish, Marathi, Marathi writtent in English and English itself. You know more about languages and translating them into English."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature
    )

    translation = response.choices[0].message.content.strip()
    return translation

  except Exception as e:
    print(f"Error: {str(e)}")
    return None





def test_translations(comments=None, models_to_test=None, prompts_to_test=None):
    """
    Test different model and prompt combinations

    Args:
        comments: List of comments to test (default: SAMPLE_COMMENTS[:5])
        models_to_test: List of model names to test (default: all)
        prompts_to_test: List of prompt types to test (default: all)
    """
    if comments is None:
        comments = SAMPLE_COMMENTS

    if models_to_test is None:
        models_to_test = list(MODELS.keys())

    if prompts_to_test is None:
        prompts_to_test = list(PROMPTS.keys())

    print("-"*30)
    print("GROQ TRANSLATION TEST")
    print("\n")
    print(f"Testing {len(comments)} comments")
    print(f"Models: {', '.join(models_to_test)}")
    print(f"Prompts: {', '.join(prompts_to_test)}")
    print("-"*30)

    for i, comment in enumerate(comments, 1):
        print(f"COMMENT #{i}")
        print("\n")
        print(f"Original: {comment}")
        print(f"\n{'-'*40}\n")

        for model_name in models_to_test:
            model_id = MODELS[model_name]
            print(f"MODEL: {model_name}")
            print(f"{'-'*40}")

            for prompt_name in prompts_to_test:
                prompt_template = PROMPTS[prompt_name]
                translation = translate_groq(comment, model_id, prompt_template)

                print(f"  [{prompt_name.upper()}]: {translation}")

            print()

        print(f"{'='*80}\n")



def quick_test(comment, show_all_models=False):
    """
    Quick test of a single comment with default prompt

    Args:
        comment: Single comment to translate
        show_all_models: If True, test all models. If False, only test llama3-70b
    """
    print("-"*30)
    print("QUICK TRANSLATION TEST")
    print("-"*30)
    print(f"Original: {comment}\n")

    models = MODELS if show_all_models else {"gpt-oss-20b": MODELS["gpt-oss-20b"],}

    for model_name, model_id in models.items():
        translation = translate_groq(comment, model_id, PROMPTS["political"])
        print(f"[{model_name}]: {translation}\n")



def compare_prompts(comment, model_name):
    """
    Compare different prompts on the same comment with one model

    Args:
        comment: Comment to translate

    """
    print("-"*30)
    print("PROMPT COMPARISON TEST")
    print("\n")
    print(f"Model: {model_name}")
    print(f"Original: {comment}\n")

    model_id = MODELS[model_name]

    for prompt_name, prompt_template in PROMPTS.items():
        translation = translate_groq(comment, model_id, prompt_template)
        print(f"[{prompt_name.upper()}]:")
        print(f"  {translation}\n")


In [ ]:
# Test 1: Quick single comment test
print("\n TEST 1: Quick Single Comment Test\n")
quick_test(SAMPLE_COMMENTS[3], show_all_models= True)


 TEST 1: Quick Single Comment Test

------------------------------
QUICK TRANSLATION TEST
------------------------------
Original: It's true what you said. If Justice doesn't happen here from it will promote mob justice which is even more terrible and Swift. Spot peh hi Justice deneka kya.

[gpt-oss-120b]: What you said is true. If justice isn’t served here, it will encourage mob justice, which is even more terrible and swift. Giving justice on the spot, that’s what they call justice.

[llama3.3-70b]: It's true what you said, if justice doesn't happen here, it will promote mob justice, which is even more terrible and swift, we should give justice on the spot itself.

[llama3.1-8b]: "It's true what you said. If justice doesn't happen here, it will promote mob justice, which is even more terrible and swift. Justice should be delivered right on the spot."

[gpt-oss-20b]: You are right. If justice isn’t delivered here, it will lead to mob justice, which is even more terrible and swift. Wh

In [ ]:
# Test 2: Compare prompts on one comment
print("\n TEST 2: Prompt Comparison\n")
compare_prompts(SAMPLE_COMMENTS[3], "llama3.1-8b")


 TEST 2: Prompt Comparison

------------------------------
PROMPT COMPARISON TEST


Model: llama3.1-8b
Original: It's true what you said. If Justice doesn't happen here from it will promote mob justice which is even more terrible and Swift. Spot peh hi Justice deneka kya.

[DEFAULT]:
  "It's true what you said. If justice doesn't happen here, then it will promote mob justice, which is even more terrible and swift. What's the point of justice being delivered at the spot itself?"

[POLITICAL]:
  "It's true what you said. If justice doesn't happen here, it will promote mob justice, which is even more terrible and swift. Justice should be delivered right on the spot."



In [ ]:
# Test 3: Full test of first 5 comments with 4 models and 2 prompts
print("\n TEST 3: Comprehensive Test (5 comments, 4 models, 2 prompts)\n")
test_translations(
    comments=SAMPLE_COMMENTS,
    models_to_test=["gpt-oss-120b", "llama3.3-70b", "llama3.1-8b", "gpt-oss-20b"],
    prompts_to_test=["default", "political"]
)


 TEST 3: Comprehensive Test (5 comments, 4 models, 2 prompts)

------------------------------
GROQ TRANSLATION TEST


Testing 5 comments
Models: gpt-oss-120b, llama3.3-70b, llama3.1-8b, gpt-oss-20b
Prompts: default, political
------------------------------
COMMENT #1


Original: 1:58 - Beta umar se pehly bara hogya :expressionless_face::expressionless_face:

----------------------------------------

MODEL: gpt-oss-120b
----------------------------------------
  [DEFAULT]: 1:58 - Son, you grew up before your age :expressionless_face::expressionless_face:
  [POLITICAL]: 1:58 – Son, you’ve become grown up before your age :expressionless_face::expressionless_face:

MODEL: llama3.3-70b
----------------------------------------
  [DEFAULT]: "1:58 - Son, he has become old before his age :expressionless_face::expressionless_face:"
  [POLITICAL]: At 1:58 - The kid has grown up before his age :expressionless_face::expressionless_face:

MODEL: llama3.1-8b
----------------------------------------


In [ ]:
gpt4_mini_fewshot = [
    {
        "role": "system",
        "content": (
            "You are a careful classification model for short YouTube comments about a specific "
            "road accident case in India. Your only job is to map each comment to exactly ONE of 7 "
            "predefined labels. This is for a research project, so accuracy is critical.\n\n"
            "OUTPUT FORMAT (very important):\n"
            "- Respond with ONLY the label number (an integer from 1 to 7).\n"
            "- Do NOT add spaces, text, punctuation, quotes, or explanations.\n"
            "- Example of a valid response: 3\n"
            "- Examples of INVALID responses: 'Label 3', '3\\nReason: ...', ' 3 '"
        )
    },
    {
        "role": "user",
        "content": f"""
            You will receive ONE YouTube comment at a time about a specific accident case in India.

            Comment:
            {text}

            The comment may be in English, Hinglish, Hindi, Marathi, or code-mixed, and may contain emojis.

            You must assign it to EXACTLY ONE of these 7 labels:

            1 - Judicial Accountability and Policy Demands
                Comments focused on courts, judges, legal outcomes, punishment severity, or demands for stricter laws / legal action.

            2 - Public Safety
                Concerns about road safety, traffic rules, reckless driving, drunk driving, minors driving, or how to prevent similar accidents.

            3 - Socioeconomic Privilege
                Comments about rich vs poor, power, influence, money buying the law, unequal treatment in the justice system.

            4 - Victim Sympathy
                Condolences, prayers, or emotional support for victims and their families (e.g., “RIP”, “justice for X and Y”, “feel sad for them”).

            5 - Anger or Outrage
                Strong negative emotions or frustration about the incident or its handling (towards the driver, system, society, etc.), but NOT mainly about:
                - courts / legal decisions (then use 1), or
                - rich/poor privilege (then use 3).

            6 - Irrelevant/General Comments
                - Only emojis or mostly emojis
                - Jokes, memes, or one-word reactions with no clear link to the case
                - Off-topic comments
                - Comments too vague/short to clearly map to 1–5 or 7
                When in doubt about relevance, choose 6.

            7 - Views on Similar Cases in the Past
                Mentions or questions about other past cases or comparisons (e.g., “like the Cameron Herrin case”).

            FEW-SHOT EXAMPLES (for guidance):

            Input: "The court must & should give capital punishment then only things will come to order"
            Output: 1

            Input: ":fire::victory_hand::OK_hand::hundred_points::crossed_fingers::thumbs_up:"
            Output: 6

            Input: ":red_heart::smiling_face:"
            Output: 6

            Input: "Does anyone know \"Cameron Herrin\" case ?????"
            Output: 7

            Input: "Money can buy law and everything"
            Output: 3

            Input: "Prateek bhai as per new RTO rules 'if a minor is caught driving a vehicle his parent is supposed to be punished'... ANDHA KANOON"
            Output: 2   (public safety & rules)

            Input: "Justice for Ashwini and Aneesh"
            Output: 4

            Input: "Judiciary,,,,,,moye moye ho gya"
            Output: 5

            DECISION RULES:
            - Pick the SINGLE most central theme of the comment.
            - If it is unclear or mostly irrelevant, choose 6.
            - Do NOT output anything except ONE integer from 1 to 7.

            Now, considering all of the above, output ONLY the label number for the given comment.
            """
    }
]

In [ ]:
gpt4_mini_zeroshot = [
    {
        "role": "system",
        "content": (
            "You are a precise labeler for YouTube comments about a specific road accident case in India. "
            "You must map each comment to exactly one label out of 7.\n\n"
            "OUTPUT FORMAT (strict):\n"
            "- Respond with ONLY the label number (1–7).\n"
            "- No spaces, no extra text, no punctuation, no explanation."
        )
    },
    {
        "role": "user",
        "content": f"""
            Classify this YouTube comment into EXACTLY ONE of the 7 labels below:

            Comment:
            {text}

            The comment may be in English, Hindi, Marathi, Hinglish, code-mixed, and may contain emojis.

            LABELS:

            1 - Judicial Accountability and Policy Demands
                Focus on courts, judges, legal outcomes, punishments, or demands for stricter laws/legal action.

            2 - Public Safety
                Focus on road safety, rules, reckless/drunk driving, minors driving, or preventive measures.

            3 - Socioeconomic Privilege
                Focus on rich vs poor, money or power influencing justice, unequal legal consequences.

            4 - Victim Sympathy
                Sympathy, prayers, RIP, support or solidarity for victims and their families.

            5 - Anger or Outrage
                Expressions of anger/frustration about the incident or its handling (driver, system, society), not mainly about courts (1) or privilege (3).

            6 - Irrelevant/General Comments
                Off-topic, unclear, meme-like, or mostly emojis; generic reactions without clear link to law, safety, privilege, sympathy, anger, or past cases.
                If the comment is ambiguous or too vague, choose 6.

            7 - Views on Similar Cases in the Past
                Mentions/questions/comparisons to previous similar cases.

            INSTRUCTIONS:
            - Choose the SINGLE most appropriate label based on context.
            - If relevance is weak or unclear, choose 6.
            - Reply with ONLY the label number (1–7), nothing else.
            """
    }
]


In [ ]:
llama_fewshot = f"""
<s>[INST] <<SYS>>
You are a careful classifier for short YouTube comments about a specific accident case in India.
Your task is to assign each comment to EXACTLY ONE of 7 labels.

OUTPUT FORMAT (strict):
- Respond with ONLY the label number (1–7).
- Do NOT add any other characters, spaces, or explanation.
<</SYS>>

You will receive ONE comment at a time.

Comment:
{text}

The comment may be in English, Hindi, Marathi, Hinglish, or code-mixed, and may contain emojis.

LABELS:
1 - Judicial Accountability and Policy Demands
    Courts, judges, legal outcomes, punishments, calls for stricter laws or legal action.

2 - Public Safety
    Road safety, traffic rules, reckless/drunk driving, minors driving, prevention of future accidents.

3 - Socioeconomic Privilege
    Rich vs poor, money/power influencing justice, unequal legal treatment.

4 - Victim Sympathy
    Condolences, prayers, emotional support for victims/families (e.g., "RIP", "justice for X and Y").

5 - Anger or Outrage
    Strong anger or frustration about the incident or system, but not mainly about:
    - courts/legal process (then use 1), or
    - rich/poor privilege (then use 3).

6 - Irrelevant/General Comments
    Emojis only, memes, jokes, off-topic, or comments too vague to fit clearly into 1–5 or 7.
    When in doubt about relevance, choose 6.

7 - Views on Similar Cases in the Past
    Mentions/questions/comparisons to other past cases.

FEW-SHOT EXAMPLES:

Input: "The court must & should give capital punishment then only things will come to order"
Output: 1

Input: ":fire::victory_hand::OK_hand::hundred_points::crossed_fingers::thumbs_up:"
Output: 6

Input: ":red_heart::smiling_face:"
Output: 6

Input: "Does anyone know 'Cameron Herrin' case ?????"
Output: 7

Input: "Money can buy law and everything"
Output: 3

Input: "Justice for Ashwini and Aneesh"
Output: 4

Input: "Judiciary,,,,,,moye moye ho gya"
Output: 5

Now, considering all of this, output ONLY the label number (1–7) for the given comment.
[/INST]
"""


In [ ]:
llama_zeroshot = f"""
<s>[INST] <<SYS>>
You are a strict labeler for YouTube comments about a specific road accident case in India.
You must classify each comment into EXACTLY ONE of 7 predefined categories.

OUTPUT FORMAT:
- Answer with ONLY the label number (1–7).
- No spaces, no extra text, no punctuation, no explanation.
<</SYS>>

Comment:
{text}

The comment may be in English, Hindi, Marathi, Hinglish, or code-mixed, and may contain emojis.

LABELS:
1 - Judicial Accountability and Policy Demands
2 - Public Safety
3 - Socioeconomic Privilege
4 - Victim Sympathy
5 - Anger or Outrage
6 - Irrelevant/General Comments
7 - Views on Similar Cases in the Past

GUIDELINES:
- Use 1 when the focus is on courts, judges, legal decisions, punishments, or laws.
- Use 2 when the focus is road/public safety, rules, reckless driving, or preventing future accidents.
- Use 3 when the focus is rich vs poor, privilege, influence, or money buying the law.
- Use 4 for sympathy, RIP, prayers, or support for victims/families.
- Use 5 for anger or outrage not mainly about courts (1) or privilege (3).
- Use 6 for off-topic, unclear, meme/emoji-heavy, or very vague comments. When relevance is weak, choose 6.
- Use 7 if the comment refers to or compares with another past case.

Now respond with ONLY one integer from 1 to 7.
[/INST]
"""


In [ ]:
mistral_fewshot = [
    {
        "role": "system",
        "content": (
            "You are a classification assistant. You receive YouTube comments about a specific accident case in India "
            "and must assign each comment to exactly one of 7 labels. Prioritize accuracy.\n\n"
            "Output ONLY the label number (1–7). No explanations or extra text."
        )
    },
    {
        "role": "user",
        "content": f"""
            Classify the following YouTube comment:

            Comment:
            {text}

            The comment may be multi-lingual (English, Hindi, Marathi, Hinglish, code-mixed) and may include emojis.

            LABELS:
            1 - Judicial Accountability and Policy Demands
            2 - Public Safety
            3 - Socioeconomic Privilege
            4 - Victim Sympathy
            5 - Anger or Outrage
            6 - Irrelevant/General Comments
            7 - Views on Similar Cases in the Past

            Short label descriptions:
            1 → Courts, judges, legal outcomes, punishment, demands for legal action or stricter laws.
            2 → Road/public safety, traffic rules, reckless/drunk driving, minors driving, prevention.
            3 → Rich vs poor, privilege, money/power buying justice, unequal legal treatment.
            4 → Sympathy, RIP, prayers, support for victims/families.
            5 → Strong anger or frustration about the case/system, not mainly about courts (1) or privilege (3).
            6 → Off-topic, jokes, memes, generic/unclear reactions, emoji-only or mostly emojis. When relevance is doubtful, choose 6.
            7 → Mentions or comparisons to previous similar cases.

            EXAMPLES:

            Input: "The court must & should give capital punishment then only things will come to order"
            Output: 1

            Input: ":fire::victory_hand::OK_hand::hundred_points::crossed_fingers::thumbs_up:"
            Output: 6

            Input: "Does anyone know \"Cameron Herrin\" case ?????"
            Output: 7

            Input: "Money can buy law and everything"
            Output: 3

            Input: "Justice for Ashwini and Aneesh"
            Output: 4

            Input: "Judiciary,,,,,,moye moye ho gya"
            Output: 5

            INSTRUCTIONS:
            - Choose the SINGLE best label.
            - If the comment is vague or not clearly related to 1–5 or 7, classify it as 6.
            - Reply with ONLY the label number (1–7), nothing else.
            """
    }
]


In [ ]:
mistral_zeroshot = [
    {
        "role": "system",
        "content": (
            "You classify YouTube comments about a specific road accident case in India into one of 7 labels. "
            "Your response must be ONLY a single integer from 1 to 7 (the label number), with no extra text."
        )
    },
    {
        "role": "user",
        "content": f"""
            Comment:
            {text}

            Choose EXACTLY ONE label:

            1 - Judicial Accountability and Policy Demands
            2 - Public Safety
            3 - Socioeconomic Privilege
            4 - Victim Sympathy
            5 - Anger or Outrage
            6 - Irrelevant/General Comments
            7 - Views on Similar Cases in the Past

            Rules:
            - 1: Courts, judges, legal process, punishment, demands for justice or stricter laws.
            - 2: Road/public safety, reckless driving, traffic rules, minors driving, preventing future accidents.
            - 3: Class/wealth privilege, money/power influencing justice, unequal treatment.
            - 4: Sympathy or support for victims/families (including “justice for X” when clearly about empathy).
            - 5: General anger/frustration about the incident or system, not mainly about courts (1) or privilege (3).
            - 6: Off-topic, unclear, generic or meme/emoji-only comments, or weakly related reactions. When unsure, prefer 6.
            - 7: Mentions or comparisons to any previous similar case.

            Return ONLY the label number (1–7).
            """
    }
]
